In [1]:
!pip install pyspark
!pip install findspark

Could not fetch URL https://pypi.org/simple/pyspark/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pyspark/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))) - skipping


ERROR: Could not find a version that satisfies the requirement pyspark (from versions: none)
ERROR: No matching distribution found for pyspark


In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [3]:
scala_version = '2.12'  # your scala version
spark_version = '3.5.0' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Thông tin Kafka
bootstrap_servers = 'localhost:9092'
kafka_topic = 'PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7'

# Định nghĩa schema cho dữ liệu
schema = StructType([
    StructField("rating", DoubleType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("categories", StringType(), True),  # Cần xác định loại dữ liệu thích hợp
    StructField("gPlusPlaceId", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("gPlusUserId", StringType(), True)
])

# Tạo DataFrame để đọc dữ liệu từ Kafka topic
kafka_stream_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .load()

# Chuyển đổi giá trị từ JSON string sang struct với schema đã định nghĩa
json_stream_df = kafka_stream_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", schema).alias("data")) \
    .select("data.*")

In [ ]:
# Hiển thị dữ liệu nhận được từ Kafka topic
query = json_stream_df.writeStream.outputMode("append").format("console").start()

# Chờ cho quá trình stream khởi động
query.awaitTermination()
